In [215]:
import pandas as pd
import gspread
import requests

token = 'AF5y90PW7vmJALond4SEGVBFwmt3Jhj7Ba7oclAeXL4'

In [216]:
sa = gspread.service_account(filename='geprek-wani-c700b4b8002f.json')

In [217]:
sheet = sa.open('Wani Spreadsheet')

In [218]:
worksheet = sheet.worksheet('Pesanan')

In [219]:
df = pd.DataFrame(worksheet.get_all_records())

In [220]:
df.rename(columns={'Request cabe GEPREK': 'Cabe'}, inplace=True)

In [221]:
df.head()

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal
0,11/04/2023 08:12:04,JoWi,Nasi Goreng Telor,,Kulit Ayam Crispy,,
1,11/04/2023 08:42:00,Probo Krishnacahya,Indomie Telur Geprek,,,Es teh,
2,11/04/2023 08:23:21,Rey,Nasi Ayam Bakar,,Kulit Ayam Crispy,,
3,11/04/2023 08:27:58,MW,Nasi Goreng Telor,,,,
4,11/04/2023 08:38:22,Bryant A T,Indomie Telur Geprek,2,,,


In [222]:
# create a list containing 'Makanan Utama' and 'Nama'
list = ['Makanan Utama', 'Cabe', 'Nama']

# replace the empty string with 1
df[list] = df[list].replace('', 1)

df[list]

,Makanan Utama,Cabe,Nama
0,Nasi Goreng Telor,1,JoWi
1,Indomie Telur Geprek,1,Probo Krishnacahya
2,Nasi Ayam Bakar,1,Rey
3,Nasi Goreng Telor,1,MW
4,Indomie Telur Geprek,2,Bryant A T
5,Nasi Ayam Bakar,1,Abel
6,Nasi Ayam Geprek,10,Galih
7,Indomie Ayam Geprek,0,Cliffton
8,Bihun Goreng Spesial,1,Alvin
9,Indomie Ayam Geprek,1,Gavin


In [223]:
# Custom geprek
df_geprek = df[df['Makanan Utama'].str.contains('Geprek') & ~df['Makanan Utama'].str.contains('Indomie')]
df_geprek

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal
6,11/04/2023 08:39:01,Galih,Nasi Ayam Geprek,10,,,
12,11/04/2023 09:22:24,Max,Nasi Ayam Geprek,1,,,


In [224]:
# Dataframe non-geprek
df_not_geprek = df[~df['Makanan Utama'].str.contains('Geprek')]

# Dataframe geprek tanpa nasi
df_geprek_tanpa_nasi = df_geprek[df_geprek['Makanan Utama'].str.contains('tanpa nasi')]

# Dataframe geprek dengan nasi
df_geprek_dengan_nasi = df_geprek[~df_geprek['Makanan Utama'].str.contains('tanpa nasi')]

# Dataframe indomie geprek
df_indomie_geprek = df[df['Makanan Utama'].str.contains('Indomie') & df['Makanan Utama'].str.contains('Geprek')]

# Dataframe camilan
df_camilan = df[['Camilan', 'Nama']]
df_camilan = df_camilan[df_camilan['Camilan'] != '']

# Dataframe minuman
df_minuman = df[['Minuman', 'Nama']]
df_minuman = df_minuman[df_minuman['Minuman'] != '']

# Dataframe sambal
df_sambal = df[['Sambal', 'Nama']]
df_sambal = df_sambal[df_sambal['Sambal'] != '']

In [225]:
combined_prints = []
combined_prints.append(f"Pesanan Depot Wani: {pd.Timestamp.now()}")
combined_prints.append("=====================================")
combined_prints.extend([f"{sum(df_geprek_dengan_nasi['Cabe'] == i)} Nasi Ayam Geprek Cabe {i} ({', '.join(df_geprek_dengan_nasi[df_geprek_dengan_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_dengan_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_geprek_tanpa_nasi['Cabe'] == i)} Ayam Geprek tanpa nasi Cabe {i} ({', '.join(df_geprek_tanpa_nasi[df_geprek_tanpa_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_tanpa_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_not_geprek['Makanan Utama'] == i)} {i} ({', '.join(df_not_geprek[df_not_geprek['Makanan Utama'] == i]['Nama'])})" for i in df_not_geprek['Makanan Utama'].unique()])
combined_prints.extend([f"{sum(df_indomie_geprek['Cabe'] == i)} {df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Makanan Utama'].iloc[0]} Cabe {i} ({', '.join(df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Nama'])})" for i in df_indomie_geprek['Cabe'].unique()])
combined_prints.extend([f"{sum(df_camilan['Camilan'] == i)} {i} ({', '.join(df_camilan[df_camilan['Camilan'] == i]['Nama'])})" for i in df_camilan['Camilan'].unique()])
combined_prints.extend([f"{sum(df_minuman['Minuman'] == i)} {i} ({', '.join(df_minuman[df_minuman['Minuman'] == i]['Nama'])})" for i in df_minuman['Minuman'].unique()])
combined_prints.extend([f"{sum(df_sambal['Sambal'] == i)} {i} ({', '.join(df_sambal[df_sambal['Sambal'] == i]['Nama'])})" for i in df_sambal['Sambal'].unique()])

print('\n'.join(combined_prints))

Pesanan Depot Wani: 2023-04-11 12:06:16.458915
1 Nasi Ayam Geprek Cabe 10 (Galih)
1 Nasi Ayam Geprek Cabe 1 (Max)
2 Nasi Goreng Telor (JoWi, MW)
2 Nasi Ayam Bakar (Rey, Abel)
1 Bihun Goreng Spesial (Alvin)
1 Nasi Goreng Spesial (Ello Musk)
1 Nasi Ayam Richeese (Ello Musk)
2 Indomie Telur Geprek Cabe 1 (Probo Krishnacahya, Gavin)
1 Indomie Telur Geprek Cabe 2 (Bryant A T)
1 Indomie Ayam Geprek Cabe 0 (Cliffton)
2 Kulit Ayam Crispy (JoWi, Rey)
1 Jamur Crispy tanpa bumbu (Alvin)
2 Es teh (Probo Krishnacahya, Cliffton)


In [226]:
# remove all text after '(' in combined_prints
# combined_prints = [i.split('(')[0] for i in combined_prints]

# print('\n'.join(combined_prints))

In [227]:
payload = {'message' : '\n'.join(combined_prints)}
r = requests.post('https://notify-api.line.me/api/notify'
                , headers={'Authorization' : 'Bearer {}'.format(token)}
                , params = payload)